In [1]:
# import os
# os.environ["GOOGLE_API_KEY"] = ''
# os.environ["LANGCHAIN_TRACING_V2"] = ""
# os.environ["LANGCHAIN_API_KEY"]=""

from dotenv import load_dotenv
load_dotenv()

True

## Prompts

In [2]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Answer the following question based only on the provided context:

<context>
{context}
</context>


Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Answer the following question based only on the provided context:

<context>
{context}
</context>

Here is a question:
{input}"""

In [3]:
prompt_infos = [
{
    "name": "physics", 
    "description": "Good for answering questions about physics", 
    "prompt_template": physics_template,
},
{
    "name": "math", 
    "description": "Good for answering math questions", 
    "prompt_template": math_template
},
{
    "name": "History", 
    "description": "Good for answering history questions", 
    "prompt_template": history_template
},
{
    "name": "computer science", 
    "description": "Good for answering computer science questions", 
    "prompt_template": computerscience_template
}]

In [4]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:

\```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
\```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

## LLM

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")

c:\Users\ethan\miniconda3\envs\genai-course\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Document

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = Chroma.from_documents(documents, embedding_function)

## Destinations

In [7]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_template(template=physics_template)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()

physic_chain = create_retrieval_chain(retriever, document_chain)

In [9]:
prompt = ChatPromptTemplate.from_template(template=computerscience_template)

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector.as_retriever()

computerscience_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
general_chain = (
    ChatPromptTemplate.from_template("{input}") | llm
)

## Route to Destination

In [11]:
def route(info):
    print(info)
    if info["destination"]:
        if "physics" in info["destination"].lower():
            return physic_chain
        elif "computer science" in info["destination"].lower():
            return computerscience_chain
    else:
        return general_chain

In [12]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# destinations
# ['physics: Good for answering questions about physics',
#  'math: Good for answering math questions',
#  'History: Good for answering history questions',
#  'computer science: Good for answering computer science questions']

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
)

## Memory 

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

## Build Chain

In [14]:
chain = (router_prompt | llm | RouterOutputParser())

In [15]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

full_chain = chain | {"destination": itemgetter("destination"), "input": lambda x: x["next_inputs"]['input']} | RunnableLambda(
    route
)

## Usage

In [16]:
response = full_chain.invoke({"input": "how can langsmith help with testing?"})
print(response)

{'destination': None, 'input': 'how can langsmith help with testing?'}
content='**Langsmith** is a natural language processing (NLP) platform that can assist with testing in several ways:\n\n**1. Test Case Generation:**\n\n* Langsmith can analyze user stories and requirements to automatically generate test cases in natural language.\n* This helps testers create comprehensive test cases that cover various scenarios and edge cases.\n\n**2. Test Case Prioritization:**\n\n* Langsmith can prioritize test cases based on their importance and risk.\n* This enables testers to focus on the most critical test cases first, ensuring efficient testing.\n\n**3. Test Case Execution:**\n\n* Langsmith can integrate with test automation tools to execute test cases in natural language.\n* This simplifies test execution and reduces the need for manual intervention.\n\n**4. Test Result Analysis:**\n\n* Langsmith can analyze test results and identify patterns or anomalies.\n* This helps testers quickly ident

In [52]:
full_chain.get_graph().print_ascii()

               +-------------+                 
               | PromptInput |                 
               +-------------+                 
                      *                        
                      *                        
                      *                        
              +----------------+               
              | PromptTemplate |               
              +----------------+               
                      *                        
                      *                        
                      *                        
          +------------------------+           
          | ChatGoogleGenerativeAI |           
          +------------------------+           
                      *                        
                      *                        
                      *                        
            +--------------------+             
            | RouterOutputParser |             
            +--------------------+      